# Imports

In [1]:
import pandas as pd
import json
from urllib.request import Request, urlopen, HTTPError
import re
import datetime
from datetime import datetime as dt
from bs4 import BeautifulSoup as soup
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import tweepy
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import os
import nltk
from collections import Counter
import time
from tqdm import tqdm
import sys

consumer_key = os.getenv("TWITTER_PUBLIC_API")
consumer_secret = os.getenv("TWITTER_SECRET_KEY")
nltk.download('vader_lexicon')

addl_stopwords = [',','`', '', 'rt', 'http', 'https', 'RT', 'BTC', 'bitcoin', 'ETH', 'LTC', 'XRP', 'co', 'crypto', 'blockchain', 'cryptocurrency', 'cripto', 'litecoin']

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\cscat\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


# Begin work on ICO Projects

In [2]:
#ICO Watch List API Wrapper- Can be called with 'live', 'upcoming', and 'finished' on the end for their respective lists, otherwise calls all ICO's
#No API key needed, 1sec limit per call

class ICO_data():
    
    # Initiates the object (self) and allows variables to be set for use in any of the classes funtions
    def __init__(self):
        self.url =' https://api.icowatchlist.com/public/v1/'
        
    def get_json(self):
        '''
        Sends HTTP Request to provided url and returns a json (dictionary) object.

        Arguements: 'url' - Requires a full http address including any applicable API keys.
        '''
        request = Request(self.url, headers={'User-Agent': 'Python'})
        response = urlopen(request)
        raw_data = response.read()
        json_data = json.loads(raw_data)
        return json_data

    def get_ico_df(self):
        '''
        Performs the 'get_json()' funtion and converts it into a Pandas DataFrame
        '''
        json_data = self.get_json()
        ico_list = json_data['ico']['finished']
        df = pd.DataFrame(ico_list)
        return df


     
    
    def preprocess_data(self):
        '''
        Performs the 'get_df' function and removes erronus columns, converts time to DateTime objects and 
        converts the numbers to floats
        '''
        df = self.get_ico_df()
        
        df.drop(columns=['icowatchlist_url', 'image', 'website_link'], inplace = True)
        reordered_columns = ['Name', 'Description', 'Price(USD)', 'Start', 'End', 'ROI(Pct)', 'Timezone']
        df.rename(columns={'all_time_roi': 'ROI(Pct)',
                           'coin_symbol': 'Ticker',
                          'description': 'Description',
                           'end_time': 'End',
                           'name': 'Name',
                           'price_usd': 'Price(USD)',
                          'start_time': 'Start',
                           'timezone': 'Timezone',
                          }, inplace=True)
        df.set_index('Ticker', inplace = True)
        df=df.reindex(columns=reordered_columns)
        #Convert the strings to datetime objects
        df['Start'] = df['Start'].apply(lambda x: dt.strptime(x, "%Y-%m-%d %H:%M:%S"))
        df['End'] = df['End'].apply(lambda x: dt.strptime(x, "%Y-%m-%d %H:%M:%S"))
        df['Price(USD)'] = df['Price(USD)'].replace("NA",'0')
        df['ROI(Pct)'] = df['ROI(Pct)'].replace('NA','0%')
        #Split the price values that are over 1,000 at the ','
        df['Price(USD)'] = df['Price(USD)'].apply(lambda x: re.split(',', x))
        #Split the roi values at their ',', and '%'
        df['ROI(Pct)'] = df['ROI(Pct)'].apply(lambda x: re.split('[, %]', x))
        #Merge the strings back that are over 1,000 and turn them into floats
        try:
            df['Price(USD)'] = df['Price(USD)'].apply(lambda x: float(x[0] + x[1])) 
        #Convert the remaining strings back to floats    
        except:
            df['Price(USD)'] = df['Price(USD)'].apply(lambda x: float(x[0]))
        
        try:
            df['ROI(Pct)'] = df['ROI(Pct)'].apply(lambda x: float(x[0] + x[1])/100) 
        except:
            df['ROI(Pct)'] = df['ROI(Pct)'].apply(lambda x: float(x[0])/100)
        #Create a duration column from the start and end dates
        df["Duration"] = df['End'] - df['Start']
        return df

  

In [11]:
  
def check_cmc(project_name):
    '''
    Check Coin Market Cap for the project, if it exists capture the current price, market cap, volume, circulating supply, total supply, ATH, ATL
    '''
    cmc_base_url = 'https://coinmarketcap.com/currencies/'
    headers={'User-Agent':"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11"} 
    request=Request(cmc_base_url + project_name, headers=headers) 
    response =urlopen(request)
    soup = BeautifulSoup(response, 'html.parser')
        
        

# Test ICO functions

In [3]:
#Instantiate the object
ico = ICO_data()

In [4]:
#Call the funtion and create the dateframe
clean_df = ico.preprocess_data()
clean_df.head()

,Name,Description,Price(USD),Start,End,ROI(Pct),Timezone,Duration
Ticker,,,,,,,,
REP,Augur,Augur is a decentralized prediction market,11.74,2015-08-17 00:00:00,2015-10-01 00:00:00,19.2415,UTC+0,45 days
LSK,Lisk,It is a cryptocurrency and decentralized appli...,0.76,2016-02-22 00:00:00,2016-03-21 00:00:00,9.1437,UTC+0,28 days
DGD,Digix DAO,A Decentralized Autonomous Organization focuse...,12.22,2016-03-30 12:00:00,2016-03-30 12:00:00,2.7716,UTC+0,0 days
WAVES,Waves,Waves helps to make the launching and coordina...,0.78,2016-04-12 13:00:00,2016-05-31 13:00:00,3.1588,UTC+0,49 days
STRAT,Stratis,Stratis was developed to help organisations de...,0.30,2016-06-21 00:00:00,2016-07-26 00:00:00,40.5553,UTC+0,35 days


# Begin work on Coin Projects

In [5]:
def tokenizer(text):
    """Tokenizes text."""
    addl_stopwords = ['ann', '']
    text = word_tokenize(text)
    text = [word.lower() for word in text]
    regex = re.compile("[^a-zA-Z ]")
    text = [regex.sub('', word) for word in text]
    sw = set(stopwords.words('english') + addl_stopwords)
    lemmatizer = WordNetLemmatizer()
    text = [lemmatizer.lemmatize(word) for word in text]
    clean_text = [word for word in text if word not in sw]
    return clean_text



class Coin_data():
    
    def __init__(self):
        self.btctalk_ann_url = 'https://bitcointalk.org/index.php?board=159.0'
        self.cmc_base_url = 'https://coinmarketcap.com/currencies/'
        self.cmc_coin_url = 'https://coinmarketcap.com/all/views/all/'

    def get_cmc_coins(self):
        headers={'User-Agent':"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11"} 
        request=Request(self.cmc_coin_url, headers=headers) 
        response =urlopen(request)
        soup = BeautifulSoup(response, 'html.parser')
        coin_list = soup.findAll('a' , {'class':"currency-name-container link-secondary"})
        coins = []
        for coin in coin_list:
            coins.append(coin.text)
        return coins
        
        
    def get_new_projects(self):
        '''
        Gets a list of all new projects from Bitcointalk.org that are listed on CoinMarketCap
        '''
        #Get list of coins on Coin Market Cap
        print("Getting a list of all coins on Coin Market Cap")
        coin_list = self.get_cmc_coins()
        coin_list = [coin.lower() for coin in coin_list]
        
        #Prepare BS4 to scrape bitcointalk.org announcement page
        headers={'User-Agent':"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11"} 
        request=Request(self.btctalk_ann_url, headers=headers) 
        response =urlopen(request)
        soup = BeautifulSoup(response, 'html.parser')
        
        #Create a list of all the post on the announcement page
        links = soup.findAll('a')
        
        #Capture the url for each post
        links = [url.get('href') for url in links if 'ANN' in url.text]
        
        #For loop over each url saving the content of each page to a dict key
        print('Looping over each url saving the content of each page to a dict key')
        coins = {}
        count = 0
        for url in links:
            count += 1
            headers={'User-Agent':"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11"} 
            request=Request(url, headers=headers) 
            response =urlopen(request)
            soup = BeautifulSoup(response, 'html.parser')
            coins[count] = soup
            
        #For loop over each dict key (post html page) and captive the title, as well as search the body for key words
        print('Looping over each dict key (post html page) and captive the title, as well as search the body for key words')
        count = 1
        flags = ['gaurenteed', 'profit', 'government', 'approval', 'massive', 'mega', 'rich', 'money']
        name, rating = [],[]
        for i in range(len(coins)):
            count += 1
            scam_meter = 0

            for flag in flags:
                if flag in coins[i+1].text.lower():
                    scam_meter += 1
            name += [coins[i+1].title.text]
            rating += [scam_meter]
            
        #Create a dataframe to store the title and scam rating for each post
        df = pd.DataFrame({
            'Title':name,
            'Scam_Rating':rating,
        })
        return df
#commented out so the function will still run and return a df
    '''        
        #Extract the project name from the title
        print('Verifing if coin is on Coin Market Cap and removing those that are not')
        words, names=[], []
        for i in range(len(df)):
            words += tokenizer(df['Title'][i])
            for word in words:
                if word in coin_list:
                    cmc = 0
#                    names.append(word)
                else:
                    cmc = 1
                names.append(cmc)
#                    names.append('NA')
        print(names)           
#        df['Name'] = names
#        df = df.dropna()
'''                
        
        
        #Extract the ticker from the title
        
        
        #Extract a start date from the body of each post
        
        
        #Extract an end date from the body of each post
        
        
        #Capture the offering price for each coin
        
        
        #Identify the Algo for each project 
        


# Test the Coin functions

In [7]:
#Instantiate the object
coin = Coin_data()

In [8]:
#Call the funtion and create the dateframe
df = coin.get_new_projects()
df.head()

Getting a list of all coins on Coin Market Cap


NameError: name 'BeautifulSoup' is not defined

In [ ]:
#Tokenize each Title and check if any words match the coin_names[] if so add that name to the dataframe otherwise set the name to 'NA'
print('Verifing if coin is on Coin Market Cap and removing those that are not')
words={}
for i in range(len(df)):
    words[i]=tokenizer(df['Title'][i])
words

In [ ]:
for words in words[6]:
    print(words)

In [ ]:
for i in range(len(words)):
    for words in words[i]:
        print(words)

### Code from Azerpas (github)

In [6]:
class ico():
	def __init__(self):
		self.hpages = 3
		self.url = "https://icobench.com/icos?filterSort=rating-desc&page="
		self.s = requests.session()
		self.book = []
		self.r = None

	def scrape(self):
		log("Scraping ICO data...")
		for i in range(1,self.hpages+1):
			req = self.s.get(self.url+str(i))
			soup = BeautifulSoup.BeautifulSoup(req.text)

			for i in soup.findAll("div",{"class":"rate color5"}):
				current_ico = (i.parent.parent)
				if not "ico_data" in str(current_ico):
					continue
				for z in current_ico.findAll("div"):
					if "Start" in z.text and "End" in z.text:
						pass
					if "Start" in z.text:
						pattern = re.compile("Start:(.*)")
						start_date = re.findall(pattern,z.text)
					if "End" in z.text:
						pattern = re.compile("End:(.*)")
						end_date = re.findall(pattern,z.text)
				for z in current_ico.findAll("a"):
					if "href" in str(z):
						pattern = re.compile('href="(.*)">')
						link = re.findall(pattern,str(z))
				title = current_ico.find("a",{"class":"name"})
				pattern = re.compile("""href="/ico/(.*)">""")
				title = re.findall(pattern,str(title))
				title = title[0].replace('-',' ').title()

				for z in current_ico.findAll("div",{"class":"content"}):
					for p in z.findAll("p"):
						description = p.text
				try:
					sep = "KYC"
					description = description.split(sep,1)[0]
				except Exception as e:
					pass

				rating = float(i.text)

				start_date = start_date[0]
				end_date = end_date[0]
				link = "https://icobench.com" + link[0]
				self.book.append({'title':title,'start':start_date,'end':end_date,'description':description,'link':link,'rating':rating})

		maxi = self.book[0]['rating']
		self.r = {'weekWinner':self.book[0]['title'],'rate':maxi}
		for i in range(0,len(self.book)):
			if self.book[i]['rating'] > maxi:
				maxi = self.book[i]['rating']
				self.r['rate'] = maxi
				self.r['weekWinner'] = self.book[i]['title']
		log("Scraped successfully")


	def syntax(self):
		message = "📈 Meilleure ICO de ces 4 derniers jours: " + self.r['weekWinner'] + " avec un score de: " + str(self.r['rate']) + "/5\n\n"
		cutting = 0
		for i in range(0,len(self.book)):
			cutting += 1
			message += "\nNom: {}\nDescription: {}\nDébut: {}\nFin: {}\nLien: {}\nNote: {}\n----------------\n".format(self.book[i]['title'],self.book[i]['description'],self.book[i]['start'],self.book[i]['end'],self.book[i]['link'],self.book[i]['rating'])
			if (cutting % 3) == 0:
				message += "+++"
		# adapted to ICObench syntax
		return message

### End

In [7]:
ico = ICO_data()

In [8]:
ico_df = ico.preprocess_data()

In [9]:
ico_df.iloc[0]

Name                                                Augur
Description    Augur is a decentralized prediction market
Price(USD)                                          11.74
Start                                 2015-08-17 00:00:00
End                                   2015-10-01 00:00:00
ROI(Pct)                                          19.2415
Timezone                                            UTC+0
Duration                                 45 days 00:00:00
Name: REP, dtype: object

In [10]:
def tokenizer(text):
    """Tokenizes text."""
    text = word_tokenize(text)
    text = [word.lower() for word in text]
    regex = re.compile("[^a-zA-Z ]")
    text = [regex.sub('', word) for word in text]
    sw = set(stopwords.words('english') + addl_stopwords)
    lemmatizer = WordNetLemmatizer()
    text = [lemmatizer.lemmatize(word) for word in text]
    clean_text = [word for word in text if word not in sw]
    return clean_text

def token_count(tokens, N=10):
    """Returns the top N tokens from the frequency count"""
    return Counter(tokens).most_common(N)


# Functions for Twitter


def get_tweets_list(topic_of_tweet, num_of_tweets):
    '''
    Returns a dataframe of the most recent 'N' tweets from Twitter tokenized and counted.
    
    Arguements: `topic_of_tweet` : str; what hashtag is being searched 
                'num_of_tweets' : int; how many tweet do you want returned
    '''
    text,time, word_list, word_count=[],[],[],[]
    auth = tweepy.AppAuthHandler(consumer_key, consumer_secret)
    api = tweepy.API(auth)
    for tweet in tweepy.Cursor(api.search, q=topic_of_tweet, tweet_mode='extended').items(num_of_tweets):
        text.append(tweet.full_text),
        time.append(tweet.created_at)
    tweets_df = pd.DataFrame({'Tweet':text}, index=time)
    [word_list.append(tokenizer(text)) for text in tweets_df.Tweet]
    tweets_df['Tokens'] = word_list
    [word_count.append(token_count(token)) for token in tweets_df.Tokens]
    tweets_df['Word_Count'] = word_count
    
    return tweets_df

def twitter_sent_analysis(tweet_df):    
    tweet_sentiments, comp, pos, neg, neu = [],[],[],[],[]
    analyzer = SentimentIntensityAnalyzer()
    for tweet in tweet_df.Tweet:
        sentiment = analyzer.polarity_scores(tweet),
        comp.append(sentiment[0]["compound"]),
        pos.append(sentiment[0]["pos"]),
        neg.append(sentiment[0]["neg"]),
        neu.append(sentiment[0]["neu"]),
  
    tweet_df['Compound'] = comp
    tweet_df['Positive'] = pos
    tweet_df['Negative'] = neg
    tweet_df['Neutral'] = neu

    return tweet_df

def count(df):
    '''
    Takes a DataFrame with a "compund" column and returns a basic count of positive, neutral, and negative sentiment in a dict format
    '''
    positive_count, negative_count, neutral_count = 0,0,0
    for i in df['Compound']:
        if i >= 0.05:
            positive_count += 1
        elif i <= -0.05:
            negative_count += 1
        else:
            neutral_count += 1
    count={
        'Positive Tweets': positive_count,
        'Neutral Tweets': neutral_count,
       'Negavtive Tweets': negative_count
    }
    return count

def get_twitter_scores(topic_of_tweet, num_of_tweets):
    df = get_tweets_list(topic_of_tweet, num_of_tweets)
    df = twitter_sent_analysis(df)
    
    return df

In [19]:
aug_twittwer_df = get_twitter_df(ico_df.index[1], 10)

NameError: name 'get_twitter_df' is not defined

In [11]:
def twitter_score(df):    
    scores = []
    for ico in df.Name:
        search_term = '#' + str(ico)
        print(f"Searching and Scoring {search_term}")
        tweet_df = get_twitter_scores(search_term, 20)
        score = {ico :{
                'Compound' : tweet_df.Compound.mean(),
                'Positive' : tweet_df.Positive.mean(),
                'Negative' : tweet_df.Negative.mean(),
                'Neutral' : tweet_df.Neutral.mean(),
        }}
        scores.append(score)
        print(f"{ico} scored")
    print(f"Scoring of {len(scores)} tweet concluded, creating dataframe")
    
    return scores
    

In [12]:
test_df = ico_df[0:5]

In [13]:
test_df

,Name,Description,Price(USD),Start,End,ROI(Pct),Timezone,Duration
Ticker,,,,,,,,
REP,Augur,Augur is a decentralized prediction market,11.74,2015-08-17 00:00:00,2015-10-01 00:00:00,19.2415,UTC+0,45 days
LSK,Lisk,It is a cryptocurrency and decentralized appli...,0.76,2016-02-22 00:00:00,2016-03-21 00:00:00,9.1437,UTC+0,28 days
DGD,Digix DAO,A Decentralized Autonomous Organization focuse...,12.22,2016-03-30 12:00:00,2016-03-30 12:00:00,2.7716,UTC+0,0 days
WAVES,Waves,Waves helps to make the launching and coordina...,0.78,2016-04-12 13:00:00,2016-05-31 13:00:00,3.1588,UTC+0,49 days
STRAT,Stratis,Stratis was developed to help organisations de...,0.30,2016-06-21 00:00:00,2016-07-26 00:00:00,40.5553,UTC+0,35 days


In [23]:
x = twitter_score(test_df)

Searching and Scoring #Augur
Augur scored
Searching and Scoring #Lisk
Lisk scored
Searching and Scoring #Digix DAO
Digix DAO scored
Searching and Scoring #Waves
Waves scored
Searching and Scoring #Stratis
Stratis scored
Scoring of 5 tweet concluded, creating dataframe


In [ ]:
b=pd.DataFrame()
b.head()

In [ ]:
for item in x:
    t=pd.DataFrame.from_dict(item).T
    b = pd.concat([b,t], sort=True)
    b.head()

In [ ]:
b

In [ ]:
x ={
    'Compuond' : aug_twittwer_df['Compound'].mean(),
    'Positive' : aug_twittwer_df['Positive'].mean(),
    'Negative' : aug_twittwer_df['Negative'].mean(),
    'Neutral' : aug_twittwer_df['Neutral'].mean(),
}

In [ ]:
aug_twittwer_df.append(x, ignore_index=True)

In [ ]:
test_df = ico_df.iloc[0:3]

In [ ]:
test_df

In [ ]:
test_df['Compound'] = ''
test_df['Positive'] = ''
test_df['Negative'] = ''
test_df['Neutral'] = ''

In [ ]:
test_df

In [ ]:
test_df.Compound[1] = aug_twittwer_df.Compound.mean()
test_df.Positive[1]= aug_twittwer_df.Positive.mean()
test_df.Negative[1]= aug_twittwer_df.Negative.mean()
test_df.Neutral[1] = aug_twittwer_df.Neutral.mean()

In [ ]:
test_df

In [ ]:
test_df.Negative[1]

In [19]:
def twitter_df_score(df, N): 
    '''
    Scores an entire Dataframe of coins based on the last 'N' tweets.  Returns a dataframe of scores with a 
    '''
    scores = []
    for name in df.Name:
        search_term = '#' + str(name)
        print(f"Searching and Scoring {search_term}")
        tweet_df = get_twitter_scores(search_term, N)
        score = {name :{
                'Compound' : tweet_df.Compound.mean(),
                'Positive' : tweet_df.Positive.mean(),
                'Negative' : tweet_df.Negative.mean(),
                'Neutral' : tweet_df.Neutral.mean(),
        }}
        scores.append(score)
        print(f"{name} scored")
        for i in progressbar(range(10), "Waiting for Twitter Rate Limit: ", 40):
            time.sleep(0.6) # any calculation you need
    print(f"Scoring of {len(scores)} tweet concluded, creating dataframe")
    df2 = pd.DataFrame()
    for item in scores:
        df1=pd.DataFrame.from_dict(item).T
        df2 = pd.concat([df1,df2], sort = True)
#    ndf = pd.concat([df, df2])

    return df2

In [20]:
twitter_scores = twitter_df_score(ico_df, 20)

Searching and Scoring #Augur
Augur scored
Waiting for Twitter Rate Limit: [########################################] 10/10
Searching and Scoring #Lisk
Lisk scored
Waiting for Twitter Rate Limit: [########################################] 10/10
Searching and Scoring #Digix DAO
Digix DAO scored
Waiting for Twitter Rate Limit: [########################################] 10/10
Searching and Scoring #Waves
Waves scored
Waiting for Twitter Rate Limit: [########################################] 10/10
Searching and Scoring #Stratis
Stratis scored
Waiting for Twitter Rate Limit: [########################################] 10/10
Searching and Scoring #Tao Network
Tao Network scored
Waiting for Twitter Rate Limit: [########################################] 10/10
Searching and Scoring #Incent
Incent scored
Waiting for Twitter Rate Limit: [########################################] 10/10
Searching and Scoring #NEO
NEO scored
Waiting for Twitter Rate Limit: [########################################] 10

In [15]:
def progressbar(it, prefix="", size=60, file=sys.stdout):
    count = len(it)
    def show(j):
        x = int(size*j/count)
        file.write("%s[%s%s] %i/%i\r" % (prefix, "#"*x, "."*(size-x), j, count))
        file.flush()        
    show(0)
    for i, item in enumerate(it):
        yield item
        show(i+1)
    file.write("\n")
    file.flush()

In [241]:
for i in progressbar(range(6), "Computing: ", 40):
    time.sleep(1) # any calculation you need

Computing: [########################################] 6/6


In [21]:
twitter_scores

,Compound,Negative,Neutral,Positive
SaltMine,0.172550,0.000000,0.94575,0.05425
VANM,-0.128000,0.048000,0.95200,0.00000
Weave,0.343325,0.014350,0.88825,0.09735
PRIMARY,0.154660,0.031550,0.87340,0.09510
Blue Chip Vision,NaN,NaN,NaN,NaN
Raido Financial,NaN,NaN,NaN,NaN
Lynked.World,NaN,NaN,NaN,NaN
Pantercon,NaN,NaN,NaN,NaN
Buying.com,0.000000,0.000000,1.00000,0.00000
Football Bitcademy,NaN,NaN,NaN,NaN


In [22]:
new_d = pd.concat([ico_df, twitter_scores])

C:\Users\cscat\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [24]:
new_d = ico_df.reset_index()

In [26]:
nd = new_d.set_index('Name')

In [48]:
n = pd.concat([twitter_scores, nd], sort=True, verify_integrity=True, join_axes='index')

AssertionError: length of join_axes must be equal to 1

In [50]:
n.to_csv('./df.csv', encoding='utf-8')